In [1]:
from numpy import array
import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Bidirectional
from keras.utils import to_categorical
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.optimizers import SGD
import tensorflow as tf

from sklearn.metrics import r2_score
import pandas as pd
# from sodapy import Socrata



In [2]:
#Uncomment the below to open data source, download sensor data, and store it as a csv locally.

##Function to get Sensor count history data
# def sensor_count():
#     client = Socrata('data.melbourne.vic.gov.au', 'nlPM0PQJSjzCsbVqntjPvjB1f', None)
#     sensor_data_id = "b2ak-trbp"
#     results = client.get(sensor_data_id, limit=5000000)
#     df = pd.DataFrame.from_records(results)
#     df = df[['date_time', 'year', 'month', 'mdate', 'day', 'time', 'sensor_id', 'sensor_name', 'hourly_counts']]
#     return df

# sensor_history = sensor_count()

# sensor_history.to_csv('sensor_history.csv', index=False)

In [2]:
sensor_history = pd.read_csv('sensor_history.csv')

In [3]:
single_sensor = sensor_history[sensor_history.sensor_id == 4].copy()
single_sensor.sort_values('date_time', inplace=True, ascending=True)

In [4]:
#Make sensor counts range from 0 to 1.
maximum = single_sensor.hourly_counts.max()
single_sensor['hourly_counts'] = single_sensor['hourly_counts']/maximum

In [5]:
sensor_seq = []
for i in range(0, len(single_sensor)):
    sensor_seq.append(single_sensor.hourly_counts.iloc[i])

In [13]:
# split a univariate sequence
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

# choose a number of time steps
n_steps = 392
# split into samples
X, y = split_sequence(sensor_seq, n_steps)


In [14]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))


In [15]:
X_train = X[10001:80000]
X_val = X[80001:]
X_test = X[:10000]
Y_train = y[10001:80000]
Y_val = y[80001:]
Y_test = y[:10000]

In [20]:
epochs = 50
learning_rate = 0.00001
decay_rate = learning_rate / epochs
momentum = 0.8

# define model
model = Sequential()
model.add(LSTM(48, return_sequences=True, input_shape=(n_steps, n_features)))
model.add(Dropout(0.2))
model.add(LSTM(96, return_sequences=True, input_shape=(n_steps, n_features)))
model.add(Dropout(0.2))
model.add(LSTM(120, return_sequences=True, input_shape=(n_steps, n_features)))
model.add(Dropout(0.2))
model.add(LSTM(196, input_shape=(n_steps, n_features)))
model.add(Dropout(0.2))
model.add(Dense(48, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

opt = keras.optimizers.Adam(learning_rate=learning_rate)

model.compile(optimizer=opt, loss='mse')


In [21]:
# fit model
model.fit(X_train, Y_train, validation_data =(X_val, Y_val), epochs=45, verbose=1, batch_size = 64)


Epoch 1/45
1094/1094 [==============================] - 164s 145ms/step - loss: 0.0440 - val_loss: 0.0227
Epoch 2/45
1094/1094 [==============================] - 114s 104ms/step - loss: 0.0260 - val_loss: 0.0223
Epoch 3/45
1094/1094 [==============================] - 111s 101ms/step - loss: 0.0257 - val_loss: 0.0241
Epoch 4/45
1094/1094 [==============================] - 107s 98ms/step - loss: 0.0154 - val_loss: 0.0043
Epoch 5/45
1094/1094 [==============================] - 105s 96ms/step - loss: 0.0043 - val_loss: 0.0036
Epoch 6/45
1094/1094 [==============================] - 105s 96ms/step - loss: 0.0039 - val_loss: 0.0036
Epoch 7/45
1094/1094 [==============================] - 104s 95ms/step - loss: 0.0038 - val_loss: 0.0033
Epoch 8/45
1094/1094 [==============================] - 104s 95ms/step - loss: 0.0037 - val_loss: 0.0033
Epoch 9/45
1094/1094 [==============================] - 105s 96ms/step - loss: 0.0037 - val_loss: 0.0031
Epoch 10/45
1094/1094 [=============================

In [25]:
# demonstrate prediction
x_input = array(sensor_seq[10001:10393])
# x_input = x_input/maximum
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print("predicted next value: ", yhat * maximum)
print("actual next value: ", sensor_seq[10393] * maximum)

predicted next value:  [[127.42242]]
actual next value:  69.0


In [26]:
# demonstrate prediction
x_input = array(sensor_seq[10021:10413])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print("predicted next value: ", yhat * maximum)
print("actual next value: ", sensor_seq[10413] * maximum)

predicted next value:  [[1327.6808]]
actual next value:  1470.0


In [22]:
pred = model.predict(X_test)

313/313 [==============================] - 11s 30ms/step


In [23]:
r2_score(Y_test, pred)

0.9337978519191391

In [24]:
# model.save("model_3.h5")